In [ ]:

import os
from dotenv import load_dotenv
import pandas as pd
import panel as pn
from sqlalchemy import create_engine, text

# Inicialização do Panel e variáveis de ambiente
pn.extension()
load_dotenv()

# Conexão com o PostgreSQL
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)


#          ABA DE CONSULTA OFICINAS
# Filtros para Oficina
filtro_titulo    = pn.widgets.TextInput(
    name='Filtrar por Título', placeholder='parte do título...'
)
filtro_professor = pn.widgets.TextInput(
    name='Filtrar por Professor', placeholder='parte do nome do professor...'
)

# Botão de consulta e painel de resultados
btn_consultar   = pn.widgets.Button(name='Consultar', button_type='success')
tabela_consulta = pn.pane.DataFrame(height=300)

def consulta_oficina(event):
    df = pd.read_sql_query("""
        SELECT
          o.id_oficina   AS id,
          o.titulo,
          o.descricao,
          o.limitevagas,
          o.status,
          o.cargahoraria,
          CONCAT(p.pnome, ' ', p.sobrenome) AS professor
        FROM oficina o
        JOIN professor pr ON o.id_professor = pr.id_professor
        JOIN pessoa p ON pr.id_professor = p.id_pessoa
    """, engine)

    mask      = pd.Series(True, index=df.index)
    titulo    = filtro_titulo.value.strip()
    prof_nome = filtro_professor.value.strip()
    if titulo:
        mask &= df['titulo'].str.contains(titulo, case=False, na=False)
    if prof_nome:
        mask &= df['professor'].str.contains(prof_nome, case=False, na=False)

    tabela_consulta.object = df[mask]

btn_consultar.on_click(consulta_oficina)

aba_consulta = pn.Column(
    "# Consulta de Oficinas",
    pn.Row(filtro_titulo, filtro_professor),
    btn_consultar,
    tabela_consulta
)


#        ABA DE INSERÇÃO OFICINAS
# Widgets de entrada
titulo_in        = pn.widgets.TextInput(name='Título', placeholder='Título da oficina...', width=300)
descricao_in     = pn.widgets.TextAreaInput(name='Descrição', placeholder='Descrição detalhada...', height=100, width=300)
limite_vagas_in  = pn.widgets.IntInput(name='Limite de Vagas', value=0, width=120)
status_in        = pn.widgets.TextInput(name='Status', placeholder='Ex: Ativado/Desativado', width=150)
carga_horaria_in = pn.widgets.IntInput(name='Carga Horária (h)', value=0, width=120)

# Dropdown de professores (carrega no startup)
_prof_df = pd.read_sql_query("""
    SELECT pr.id_professor, p.pnome, p.sobrenome
    FROM professor pr
    JOIN pessoa p ON pr.id_professor = p.id_pessoa
""", engine)
prof_options = {
    f"{row['id_professor']} - {row['pnome']} {row['sobrenome']}": row['id_professor']
    for _, row in _prof_df.iterrows()
}
professor_select = pn.widgets.Select(name='Professor', options=prof_options, width=300)

btn_inserir_off = pn.widgets.Button(name='Inserir Oficina', button_type='success')
mensagem_ins_off = pn.pane.Markdown("")

def inserir_oficina(event):
    erros = []
    titulo    = titulo_in.value.strip()
    descricao = descricao_in.value.strip() or None
    vagas     = limite_vagas_in.value
    status    = status_in.value.strip()
    carga     = carga_horaria_in.value
    prof_id   = professor_select.value

    # Validações
    if not titulo:
        erros.append("Título é obrigatório.")
    if vagas is None or vagas < 0:
        erros.append("Limite de vagas deve ser ≥ 0.")
    if not status:
        erros.append("Status é obrigatório.")
    if carga is None or carga < 0:
        erros.append("Carga horária deve ser ≥ 0.")
    if prof_id is None:
        erros.append("Selecione um professor.")

    if erros:
        mensagem_ins_off.object = "\n".join(f"❌ {e}" for e in erros)
        return

    with engine.begin() as conn:
        new_id = conn.execute(text("""
            INSERT INTO oficina (titulo, descricao, limiteVagas, status, cargaHoraria, id_professor)
            VALUES (:t, :d, :v, :s, :c, :pid)
            RETURNING id_oficina
        """), {
            't': titulo,
            'd': descricao,
            'v': vagas,
            's': status,
            'c': carga,
            'pid': prof_id
        }).scalar()

    mensagem_ins_off.object = f"✅ Oficina inserida com sucesso (id={new_id})"
    titulo_in.value = ''
    descricao_in.value = ''
    limite_vagas_in.value = 0
    status_in.value = ''
    carga_horaria_in.value = 0

btn_inserir_off.on_click(inserir_oficina)

aba_insercao = pn.Column(
    "# Inserção de Oficinas",
    titulo_in,
    descricao_in,
    pn.Row(limite_vagas_in, carga_horaria_in, status_in),
    professor_select,
    pn.Row(btn_inserir_off, mensagem_ins_off)
)

#      ABA DE REMOÇÃO DE OFICINAS
id_officina_in   = pn.widgets.IntInput(name='ID Oficina', placeholder='Digite o ID...', width=150)
btn_remover_off  = pn.widgets.Button(name='Remover Oficina', button_type='danger')
mensagem_rem_off = pn.pane.Markdown("")

def remover_oficina(event):
    rid = id_officina_in.value
    if not rid:
        mensagem_rem_off.object = "❌ ID inválido."
        return

    with engine.begin() as conn:
        exists = conn.execute(
            text("SELECT 1 FROM oficina WHERE id_oficina = :rid"),
            {'rid': rid}
        ).fetchone()

    if not exists:
        mensagem_rem_off.object = f"❌ Oficina com ID={rid} não encontrada."
        return

    try:
        with engine.begin() as conn:
            conn.execute(text("DELETE FROM oficina_categorias      WHERE id_oficina = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM favoritooficina         WHERE id_oficina = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM listamatricula          WHERE id_oficina = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM historicoparticipacao   WHERE id_oficina = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM avaliacao              WHERE id_oficina = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM aula                   WHERE id_oficina = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM oficina                WHERE id_oficina = :rid"), {'rid':rid})
        mensagem_rem_off.object = f"✅ Oficina ID={rid} removida com sucesso."
    except Exception as e:
        mensagem_rem_off.object = f"❌ Erro ao remover oficina: {e}"

btn_remover_off.on_click(remover_oficina)

aba_remocao = pn.Column(
    "# Remoção de Oficinas",
    id_officina_in,
    btn_remover_off,
    mensagem_rem_off
)

#      MONTA A APLICAÇÃO FINAL

app = pn.Tabs(
    ('Consulta Oficinas', aba_consulta),
    ('Inserção Oficinas', aba_insercao),
    ('Remoção Oficinas', aba_remocao)
)
app.servable()
